In [1]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

class BitstringPRNG:
    def __init__(self, seed: bytes = None):
        """
        Initialize the PRNG with an optional 32-byte seed.
        If not provided, a random seed is generated.
        """
        if seed is None:
            seed = os.urandom(32)
        if len(seed) != 32:
            raise ValueError("Seed must be exactly 32 bytes (256 bits)")
        self.key = seed
        self.nonce = b'\x00' * 16  # Fixed nonce for deterministic output
        self.counter = 0

    def _generate_block(self) -> bytes:
        """
        Generate a 64-byte block of pseudorandom data using ChaCha20.
        """
        # Construct a nonce with a counter to avoid repetition
        counter_nonce = self.nonce[:8] + self.counter.to_bytes(8, 'little')
        algorithm = algorithms.ChaCha20(self.key, counter_nonce)
        cipher = Cipher(algorithm, mode=None, backend=default_backend())
        encryptor = cipher.encryptor()
        keystream = encryptor.update(b'\x00' * 64)  # Encrypt null bytes to get keystream
        self.counter += 1
        return keystream

    def get_bits(self, n_bits: int) -> str:
        """
        Generate a pseudorandom bitstring of length `n_bits`.
        """
        result = bytearray()
        while len(result) * 8 < n_bits:
            result.extend(self._generate_block())

        # Convert to bitstring and truncate
        bits = ''.join(f'{byte:08b}' for byte in result)
        return bits[:n_bits]

# === Example usage ===
if __name__ == "__main__":
    prng = BitstringPRNG()  # Use random seed
    bitstring_128 = prng.get_bits(128)
    bitstring_256 = prng.get_bits(256)

    print("128-bit pseudorandom bitstring:", bitstring_128)
    print("256-bit pseudorandom bitstring:", bitstring_256)


128-bit pseudorandom bitstring: 10011001011001101001111100010010010110001111111111010010111101001100110111001000111010000111001101100100010011101110011100111000
256-bit pseudorandom bitstring: 0000011001100000001110011000110001111110110110010001101111100101111010001111010001001000110000110010000101101001100001000110000011010011010101110000010110001110100110101100010011000110011101100011010001011110111100010000011001111111001100010111100100101111


In [2]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

class BitstringPRNG:
    def __init__(self, seed: bytes = None):
        if seed is None:
            seed = os.urandom(32)
        if len(seed) != 32:
            raise ValueError("Seed must be exactly 32 bytes")
        self.key = seed
        self.nonce = b'\x00' * 16
        self.counter = 0

    def _generate_block(self) -> bytes:
        counter_nonce = self.nonce[:8] + self.counter.to_bytes(8, 'little')
        algorithm = algorithms.ChaCha20(self.key, counter_nonce)
        cipher = Cipher(algorithm, mode=None, backend=default_backend())
        encryptor = cipher.encryptor()
        stream = encryptor.update(b'\x00' * 64)
        self.counter += 1
        return stream

    def get_keystream(self, length: int) -> bytes:
        out = bytearray()
        while len(out) < length:
            out.extend(self._generate_block())
        return bytes(out[:length])

def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))

def bytes_to_bitstring(data: bytes) -> str:
    return ''.join(f'{b:08b}' for b in data)

# === Example usage ===
if __name__ == "__main__":
    # Message to encrypt
    message = b"Stream cipher with PRNG XOR!"
    print("Original message        :", message)
    print("Original message (bits) :", bytes_to_bitstring(message))

    # Initialize PRNG with fixed seed for repeatability
    seed = os.urandom(32)
    prng_encrypt = BitstringPRNG(seed)
    keystream = prng_encrypt.get_keystream(len(message))

    ciphertext = xor_bytes(message, keystream)
    print("\nCiphertext (bits)       :", bytes_to_bitstring(ciphertext))

    # Decryption (use same seed!)
    prng_decrypt = BitstringPRNG(seed)
    keystream2 = prng_decrypt.get_keystream(len(ciphertext))
    decrypted = xor_bytes(ciphertext, keystream2)

    print("\nDecrypted message (bits):", bytes_to_bitstring(decrypted))
    print("Decrypted message       :", decrypted)


Original message        : b'Stream cipher with PRNG XOR!'
Original message (bits) : 01010011011101000111001001100101011000010110110100100000011000110110100101110000011010000110010101110010001000000111011101101001011101000110100000100000010100000101001001001110010001110010000001011000010011110101001000100001

Ciphertext (bits)       : 11110110011010101011110101100111110111111110111111110110111001101010011010010101100011001001110000001000011001100000010110110001010011000111011101000100100100101100111101111111011110011101010000001110011000000100111101010000

Decrypted message (bits): 01010011011101000111001001100101011000010110110100100000011000110110100101110000011010000110010101110010001000000111011101101001011101000110100000100000010100000101001001001110010001110010000001011000010011110101001000100001
Decrypted message       : b'Stream cipher with PRNG XOR!'


The XOR-based scheme here is only as secure as the PRNG.

Using ChaCha20-based PRNG with a unique seed per message is considered secure (if the seed is secret).

This is equivalent to OTP (One-Time Pad) if the keystream is never reused.